# RNAi Phenotypes Wormbase (WS264)

Author: Zachary Flamholz  
Date: 06-2018  
Database: https://wormbase.org/#012-34-5  
Data: ftp://ftp.wormbase.org/pub/wormbase/releases/WS264/ONTOLOGY/rnai_phenotypes.WS264.wb  
Companion file: http://ftp.ncbi.nlm.nih.gov/gene/DATA/GENE_INFO/Invertebrates/Caenorhabditis_elegans.gene_info.gz,  http://tazendra.caltech.edu/~azurebrd/cgi-bin/forms/generic.cgi?action=WpaXref  

# Versions of Modules in Use

In [1]:
%load_ext version_information
%version_information numpy, pandas, clustergrammer_widget 

Software versions
Python 2.7.15 64bit [GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
IPython 5.7.0
OS Darwin 17.5.0 x86_64 i386 64bit
numpy 1.14.3
pandas 0.23.0
clustergrammer_widget The 'clustergrammer_widget' distribution was not found and is required by the application
Tue Jul 10 11:46:04 2018 EDT

## import libraries

In [2]:
import numpy as np
import pandas as pd
import sys
import datetime
import scipy.stats as stat

## read in data

In [3]:
df = pd.read_csv("input/rnai_phenotypes.WS264_WormBase.txt", sep="\t", header=None)
df.columns = ["WB gene ID", "Gene symbol", "phenotype", "WB phenotype ID", "WB source ID"]

In [4]:
df.head()

,WB gene ID,Gene symbol,phenotype,WB phenotype ID,WB source ID
0,WBGene00001908,F17E9.9,locomotion variant,WBPhenotype:0000643,WBRNAi00025129|WBPaper00006395 WBRNAi00025631|...
1,WBGene00001908,F17E9.9,avoids bacterial lawn,WBPhenotype:0000402,WBRNAi00095640|WBPaper00040984
2,WBGene00001908,F17E9.9,RAB-11 recycling endosome localization variant,WBPhenotype:0002107,WBRNAi00090830|WBPaper00041129
3,WBGene00001908,F17E9.9,body elongation defective,WBPhenotype:0000242,WBRNAi00082054|WBPaper00005085
4,WBGene00001908,F17E9.9,chromosome segregation variant,WBPhenotype:0000773,WBRNAi00082059|WBPaper00005085


In [5]:
df.shape

(54540, 5)

In [6]:
len(df["phenotype"].unique())

1219

In [7]:
len(df["Gene symbol"].unique())

8076

In [8]:
df_location_variant = df.loc[df["phenotype"] == "locomotion variant"].copy()

In [9]:
df_location_variant.iloc[0]

WB gene ID                                            WBGene00001908
Gene symbol                                                  F17E9.9
phenotype                                         locomotion variant
WB phenotype ID                                  WBPhenotype:0000643
WB source ID       WBRNAi00025129|WBPaper00006395 WBRNAi00025631|...
Name: 0, dtype: object

In [10]:
df_location_variant.iloc[0]["WB gene ID"]

'WBGene00001908'

## analyze term set

In [11]:
count_terms = lambda x: len(df.loc[df["phenotype"] == x])
func = np.vectorize(count_terms)
numGenes_perTerms = func(df["phenotype"].unique())


In [12]:
np.mean(numGenes_perTerms)

44.741591468416736

In [13]:
np.median(numGenes_perTerms)

6.0

## convert gene symbol column to general nomenclature

In [14]:
cElegans_geneInfo = pd.read_csv("input/Caenorhabditis_elegans.gene_info-1", sep="\t")

In [15]:
cElegans_geneInfo.head()

,#tax_id,GeneID,Symbol,LocusTag,Synonyms,dbXrefs,chromosome,map_location,description,type_of_gene,Symbol_from_nomenclature_authority,Full_name_from_nomenclature_authority,Nomenclature_status,Other_designations,Modification_date,Feature_type
0,6239,171590,homt-1,CELE_Y74C9A.3,-,WormBase:WBGene00022277,I,-,Alpha N-terminal protein methyltransferase 1,protein-coding,-,-,-,Alpha N-terminal protein methyltransferase 1,20180606,-
1,6239,171591,nlp-40,CELE_Y74C9A.2,-,WormBase:WBGene00022276,I,-,Peptide P4,protein-coding,-,-,-,Peptide P4,20180606,-
2,6239,171592,rcor-1,CELE_Y74C9A.4,-,WormBase:WBGene00022278,I,-,RCOR (REST CO-Repressor) homolog,protein-coding,-,-,-,RCOR (REST CO-Repressor) homolog,20180606,-
3,6239,171593,sesn-1,CELE_Y74C9A.5,-,WormBase:WBGene00022279,I,-,Sestrin homolog,protein-coding,-,-,-,Sestrin homolog,20180606,-
4,6239,171594,pgs-1,CELE_Y48G1C.4,-,WormBase:WBGene00021677,I,-,CDP-diacylglycerol--glycerol-3-phosphate 3-pho...,protein-coding,-,-,-,CDP-diacylglycerol--glycerol-3-phosphate 3-pho...,20180406,-


In [16]:
cElegans_geneInfo.iloc[0,5].split(":")[1]

'WBGene00022277'

In [17]:
remove_WormBase = lambda x: x.replace("|", ":").split(":")[1] if "WormBase" in x else x
## there are two rows that have muliple dbXrefs 
cElegans_geneInfo["dbXrefs"] = cElegans_geneInfo["dbXrefs"].apply(remove_WormBase)
cElegans_geneInfo.head()

,#tax_id,GeneID,Symbol,LocusTag,Synonyms,dbXrefs,chromosome,map_location,description,type_of_gene,Symbol_from_nomenclature_authority,Full_name_from_nomenclature_authority,Nomenclature_status,Other_designations,Modification_date,Feature_type
0,6239,171590,homt-1,CELE_Y74C9A.3,-,WBGene00022277,I,-,Alpha N-terminal protein methyltransferase 1,protein-coding,-,-,-,Alpha N-terminal protein methyltransferase 1,20180606,-
1,6239,171591,nlp-40,CELE_Y74C9A.2,-,WBGene00022276,I,-,Peptide P4,protein-coding,-,-,-,Peptide P4,20180606,-
2,6239,171592,rcor-1,CELE_Y74C9A.4,-,WBGene00022278,I,-,RCOR (REST CO-Repressor) homolog,protein-coding,-,-,-,RCOR (REST CO-Repressor) homolog,20180606,-
3,6239,171593,sesn-1,CELE_Y74C9A.5,-,WBGene00022279,I,-,Sestrin homolog,protein-coding,-,-,-,Sestrin homolog,20180606,-
4,6239,171594,pgs-1,CELE_Y48G1C.4,-,WBGene00021677,I,-,CDP-diacylglycerol--glycerol-3-phosphate 3-pho...,protein-coding,-,-,-,CDP-diacylglycerol--glycerol-3-phosphate 3-pho...,20180406,-


## testing how to convert names

In [18]:
type(cElegans_geneInfo["dbXrefs"][0])

str

In [19]:
type(df.iloc[0,0])

str

In [20]:
cElegans_geneInfo.loc[cElegans_geneInfo["dbXrefs"] == df.iloc[0,0]]["Symbol"]

17913    his-34
Name: Symbol, dtype: object

In [21]:
'WBGene00022277' in cElegans_geneInfo["dbXrefs"].values

True

In [22]:
df_location_variant.head(10)

,WB gene ID,Gene symbol,phenotype,WB phenotype ID,WB source ID
0,WBGene00001908,F17E9.9,locomotion variant,WBPhenotype:0000643,WBRNAi00025129|WBPaper00006395 WBRNAi00025631|...
57,WBGene00007110,B0035.11,locomotion variant,WBPhenotype:0000643,WBRNAi00024350|WBPaper00006395
118,WBGene00006707,R09B3.4,locomotion variant,WBPhenotype:0000643,WBRNAi00026070|WBPaper00006395
168,WBGene00003808,T05A1.1,locomotion variant,WBPhenotype:0000643,WBRNAi00075428|WBPaper00006132
185,WBGene00016250,C30C11.4,locomotion variant,WBPhenotype:0000643,WBRNAi00024683|WBPaper00006395
226,WBGene00012735,Y40B1A.4,locomotion variant,WBPhenotype:0000643,WBRNAi00004607|WBPaper00004402 WBRNAi00026682|...
289,WBGene00004951,K10B3.10,locomotion variant,WBPhenotype:0000643,WBRNAi00008979|WBPaper00005654 WBRNAi00025934|...
355,WBGene00017678,F21F8.4,locomotion variant,WBPhenotype:0000643,WBRNAi00078339|WBPaper00027214
393,WBGene00019778,M57.2,locomotion variant,WBPhenotype:0000643,WBRNAi00009019|WBPaper00005654 WBRNAi00026001|...
535,WBGene00019881,R05D3.8,locomotion variant,WBPhenotype:0000643,WBRNAi00026026|WBPaper00006395 WBRNAi00074594|...


In [23]:
type(cElegans_geneInfo.loc[cElegans_geneInfo["dbXrefs"] == 'WBGene00001908']["Symbol"].tolist()[0])

str

In [24]:
## if there is no WB gene ID in the reference table then the symbol becomes the WB gene ID from the phenotype list
change_name = lambda x: cElegans_geneInfo.loc[cElegans_geneInfo["dbXrefs"] == x]["Symbol"].tolist()[0] if x in cElegans_geneInfo["dbXrefs"].values else x
df_location_variant["Gene symbol"] = df_location_variant["WB gene ID"].apply(change_name)

In [25]:
df_location_variant.head(100)

,WB gene ID,Gene symbol,phenotype,WB phenotype ID,WB source ID
0,WBGene00001908,his-34,locomotion variant,WBPhenotype:0000643,WBRNAi00025129|WBPaper00006395 WBRNAi00025631|...
57,WBGene00007110,leo-1,locomotion variant,WBPhenotype:0000643,WBRNAi00024350|WBPaper00006395
118,WBGene00006707,ubc-12,locomotion variant,WBPhenotype:0000643,WBRNAi00026070|WBPaper00006395
168,WBGene00003808,npr-2,locomotion variant,WBPhenotype:0000643,WBRNAi00075428|WBPaper00006132
185,WBGene00016250,hsp-110,locomotion variant,WBPhenotype:0000643,WBRNAi00024683|WBPaper00006395
226,WBGene00012735,sptf-3,locomotion variant,WBPhenotype:0000643,WBRNAi00004607|WBPaper00004402 WBRNAi00026682|...
289,WBGene00004951,spc-1,locomotion variant,WBPhenotype:0000643,WBRNAi00008979|WBPaper00005654 WBRNAi00025934|...
355,WBGene00017678,asp-12,locomotion variant,WBPhenotype:0000643,WBRNAi00078339|WBPaper00027214
393,WBGene00019778,M57.2,locomotion variant,WBPhenotype:0000643,WBRNAi00009019|WBPaper00005654 WBRNAi00026001|...
535,WBGene00019881,R05D3.8,locomotion variant,WBPhenotype:0000643,WBRNAi00026026|WBPaper00006395 WBRNAi00074594|...


## use reference list to extract only protein coding genes

In [26]:
cElegans_proteinCoding = cElegans_geneInfo[cElegans_geneInfo["type_of_gene"] == "protein-coding"]

In [27]:
df_onlyProteinCoding = df.loc[df["WB gene ID"].isin(cElegans_proteinCoding["dbXrefs"].unique())]

In [28]:
df_onlyProteinCoding.shape

(53073, 5)

## load the table for PMID # reference

In [29]:
paper_lookup = pd.read_csv('input/WB_paper_id_conversion_table', sep='~', header=None)
paper_lookup.columns = ['info']

In [30]:
paper_lookup.head()

,info
0,WBPaper00000003 cgc3
1,WBPaper00000005 cgc5
2,WBPaper00000005 doi10.1163/187529275X00518
3,WBPaper00000006 cgc6
4,WBPaper00000007 cgc7


In [31]:
## need to split the file and then create a dictionary for each paper id
table = paper_lookup['info'].apply(lambda x: x.split(' '))
table_dict = {}
for i in range(len(table)):
    table_dict[table[i][0]] = table[i][1]

In [32]:
table_dict['WBPaper00006395']

'pmid14551910'

## create gmt by writing to file

In [33]:
filename = 'wormbase_rnai_phenotype_greater4_%s.gmt'% str(datetime.date.today())[0:7].replace('-', '_')
file = open(filename,'w+') 
terms = df["phenotype"].unique()
for term in terms:
    df_byTerm = df_onlyProteinCoding.loc[df["phenotype"] == term].copy()
    
    if df_byTerm.shape[0] > 4:
        # convert gene name using the cElegans gene info table
        change_name = lambda x: cElegans_geneInfo.loc[cElegans_geneInfo["dbXrefs"] == x]["Symbol"].tolist()[0] if x in cElegans_geneInfo["dbXrefs"].values else x
        df_byTerm["Gene symbol"] = df_byTerm["WB gene ID"].apply(change_name)
        # split splice variant names. some genes have format X.#.# where the second decimal describes a TX ID
        split_splice = lambda x: (x.split('.')[0] + x.split('.')[1]) if len(x.split('.')) > 2 else x
        df_byTerm["Gene symbol"] = df_byTerm["Gene symbol"].apply(split_splice)
        
        if len(df_byTerm["Gene symbol"].unique()) > 4:
            file.write("%s" % term + "_" + df_byTerm.iloc[0, 3] + '\t')
            file.write("\t")
            genes = df_byTerm["Gene symbol"].unique()
            for gene in genes:
                file.write("%s\t" % gene)
            file.write("\n")
        
file.close()

## get mean and median for each term

In [34]:
def getTermStats(loaded_gmt, geneInfoTable, num_samplings, num_genes_in_sampling) :
    # get the relevant terms and set a dictionary for them
    term_set = loaded_gmt.iloc[:,0].apply(lambda x: x.split("\t")[0])
    term_genes_dict = {term_set[i]: loaded_gmt.iloc[i,0].split("\t")[2:] for i in range(0, loaded_gmt.shape[0])}
    term_rank_dict = {k: [] for k in term_set}
    term_stats_dict = {k: [] for k in term_set}
    
    # get the total number of genes in the organism
    n_genes = len(geneInfoTable["Symbol"].unique())
    
    # number of genes per sampling, number of times to sample
    genes_in_sampleing = num_genes_in_sampling
    num_samplings = num_samplings
    
    for i,x in enumerate(range(0,num_samplings)):
        
        progressPercent = ((i+1)/len(range(0,num_samplings)))*100
        
        sys.stdout.write("Progress: %d%%  %d Out of %d   \r" % (progressPercent, (i+1), len(range(0,num_samplings))))
        sys.stdout.flush()
            
        
        randomSet = set(geneInfoTable["Symbol"].sample(genes_in_sampleing))
        
        # initialize an array to hold the pvalue for each term
        pvals = np.array([])
        for key in term_rank_dict.keys():
            
            termSet = set(term_genes_dict[key])
            # calculate p value using fisher exact test
            # implemented using the formula found in the GeneOverap bioconductor package for R
            pval_term = stat.fisher_exact([[n_genes - len(termSet.union(randomSet)), len(randomSet.difference(termSet))], [len(termSet.difference(randomSet)), len(termSet.intersection(randomSet))]])[1]
            pvals = np.append(pvals, [pval_term])
        
        # sort the pvals and add the rank to the term_rank dict
        sorted_pvals = pvals.argsort()
        
        # find the rank for each term by sorting the pvals array and getting the index of the key position in the
        # sorted list. Need to +1 because the index begins with 0
        
        for j,key in enumerate(term_rank_dict.keys()):
            term_rank_dict[key].append(np.where(sorted_pvals == j)[0][0] + 1)
            
    for key in term_rank_dict:
        term_stats_dict[key] = [np.mean(term_rank_dict[key]), np.std(term_rank_dict[key])]
            
    
    
    return term_stats_dict

In [36]:
## load gmt
rnai_WB_gmt = pd.read_csv("wormbase_rnai_phenotype_greater4_2018_07.gmt", sep="~", header=None)

In [37]:
rnai_WB_gmt.head()

,0
0,locomotion variant_WBPhenotype:0000643\t\this-...
1,avoids bacterial lawn_WBPhenotype:0000402\t\th...
2,RAB-11 recycling endosome localization variant...
3,body elongation defective_WBPhenotype:0000242\...
4,chromosome segregation variant_WBPhenotype:000...


In [38]:
iterations_200 = getTermStats(rnai_WB_gmt, cElegans_proteinCoding, num_samplings=200, num_genes_in_sampling=300)

In [39]:
iterations_200

{'ABa ABp EMS synchronous division early emb_WBPhenotype:0001124': [516.39,
  198.19136686546162],
 'ATP levels reduced_WBPhenotype:0001575': [268.24, 104.2498076736835],
 'Aba ABp division axis defective early emb_WBPhenotype:0001134': [383.765,
  175.03833801484748],
 'Bacillus thuringiensis toxin hypersensitive_WBPhenotype:0002057': [245.175,
  170.17459967633243],
 'C lineage variant_WBPhenotype:0000832': [421.365, 127.49055563060348],
 'DNA damage checkpoint variant_WBPhenotype:0000741': [515.435,
  168.49734649245963],
 'DNA repair variant_WBPhenotype:0001938': [552.315, 172.32192482385983],
 'E lineage variant_WBPhenotype:0000834': [422.02, 217.01186511340802],
 'EMS anterior extension fails early emb_WBPhenotype:0001127': [273.875,
  112.85610030033821],
 'Fluorouracil resistant_WBPhenotype:0001649': [532.01, 169.46306352712972],
 'L1 arrest_WBPhenotype:0000081': [298.97, 198.90313999532538],
 'L1 lethal_WBPhenotype:0000117': [340.085, 154.1010635102821],
 'L2 arrest_WBPhenotyp

In [40]:
df_200_iterations = pd.DataFrame.from_dict(iterations_200, orient='index', columns = ['mean', 'sd'])

In [41]:
df_200_iterations.head()

,mean,sd
head notched_WBPhenotype:0000379,355.020,147.856957
transgene expression variant_WBPhenotype:0000306,219.280,155.656197
nuclear anchoring variant_WBPhenotype:0001997,422.220,151.098185
organism pathogen response variant_WBPhenotype:0001012,322.945,189.857926
embryonic terminal arrest variable emb_WBPhenotype:0000051,369.690,179.388193


In [42]:
df_200_iterations.sort_values(by=['mean']).head()

,mean,sd
organism morphology variant_WBPhenotype:0000535,154.800,84.911778
cell membrane organization biogenesis variant_WBPhenotype:0001982,156.180,81.003565
sterile_WBPhenotype:0000688,156.825,82.462018
embryonic lethal_WBPhenotype:0000050,157.910,75.544370
locomotion variant_WBPhenotype:0000643,162.520,113.118034


In [43]:
df_200_iterations.to_csv('output/wormbase_rnai_phenotype_greater4_2018_07_stats.tsv', sep='\t', header=False)